# Version
* `v13`: Fold4
* `v12`: Fold3
* `v10`: Fold2
* `v09`: Fold1
* `v03`: Fold0

In [ ]:
!pip install --upgrade seaborn
!pip install ensemble_boxes

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('max_colwidth',1600)

In [ ]:
dim = 512 #512, 256, 'original'
fold = 4

In [ ]:
train_df = pd.read_csv(f'../input/vinbigdata-{dim}-image-dataset/vinbigdata/train.csv')
train_df.head()

In [ ]:
train_df['image_path'] = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/train/'+train_df.image_id+('.png' if dim!='original' else '.jpg')
train_df.head()

# Only 14 Class

In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

In [ ]:
# train_df[(train_df.image_id=='9a5094b2563a1ef3ff50dc5c7ff71345')]

In [ ]:
# IoU = Calculate_IoU( (692.0,1375.0,1657.0,1799.0), (689.0,1313.0,1666.0,1763.0))
# print("IoU是：{}".format(IoU))

In [ ]:
# train_df[(train_df.image_id=='051132a778e61a86eb147c7c6f564dfe')]

In [ ]:
train_df.shape

In [ ]:
class_name_ids = ['Aortic enlargement','Atelectasis','Calcification','Cardiomegaly','Consolidation','ILD','Infiltration','Lung Opacity','Nodule/Mass','Other lesion','Pleural effusion','Pleural thickening','Pneumothorax','Pulmonary fibrosis']
values = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
class_dictionary = dict(zip(class_name_ids, values))
print (class_dictionary)

In [ ]:
train_df['class_id'].value_counts(normalize = False, dropna = False)

In [ ]:
# class_name_ids = ['Aortic Enlargement','Atelectasis','Calcification','Cardiomegaly','Consolidation','ILD','Infiltration','Lung Opacity','Nodule/Mass','Other Lesion','Pleural Effusion','Pleural Thickening','Pneumothorax','Pulmonary Fibrosis']
# values = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
# class_dictionary = dict(zip(class_name_ids, values))
# print (class_dictionary)

In [ ]:
# # import numpy as np
# import pandas as pd

# from tqdm import tqdm
# from ensemble_boxes import *

# consist_iou_threshold=0.3
# # ===============================
# # Default WBF config (you can change these)
# iou_thr = 0.5
# skip_box_thr = 0.0001
# sigma = 0.1
# # ===============================

# # Loading the train DF
# # df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
# df = train_df
# df.fillna(0, inplace=True)
# # df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

# results = []
# image_ids = df["image_id"].unique()


# count=3#


# for image_id in tqdm(image_ids, total=len(image_ids)):
#     count-=1
#     print('count',count)
#     if count==0:#
#         break#
    
#     print('image_id',image_id)
        
#     # All annotations for the current image.
#     data = df[df["image_id"] == image_id]
#     data = data.reset_index(drop=True)
#     annotations = {}
#     weights = []
    
    
#     width=data.iloc[0].width
#     height=data.iloc[0].height
#     image_path=data.iloc[0].image_path
#     class_name=data.iloc[0].class_name

#     # WBF expects the coordinates in 0-1 range.
#     max_value = data.iloc[:, 4:8].values.max()
#     data.loc[:, ["x_min", "y_min", "x_max", "y_max"]] = data.iloc[:, 4:8]

#     # Loop through all of the annotations
#     for idx, row in data.iterrows():

#         class_name_id = row["class_name"]

#         if class_name_id not in annotations:
#             annotations[class_name_id] = {
#                 "boxes_list": [],
#                 "rad_id_list": [],
#                 "labels_list": [],
#             }

#             # We consider all of the radiologists as equal.
#             weights.append(1.0)

#         annotations[class_name_id]["boxes_list"].append([row["x_min"], row["y_min"], row["x_max"], row["y_max"]])
#         annotations[class_name_id]["rad_id_list"].append(row["rad_id"])
#         annotations[class_name_id]["labels_list"].append(class_name_id)
#         print('class_name_id',class_name_id)

#     boxes_list = []
#     rad_list = []
#     labels_list = []
    
#     print('annotations',annotations)

#     for annotator in annotations.keys():
#         boxes_list.append(annotations[annotator]["boxes_list"])
#         rad_list.append(annotations[annotator]["rad_id_list"])
#         labels_list.append(annotations[annotator]["labels_list"])
        
#     print()
#     print('boxes_list',boxes_list)
#     print()
#     print('rad_list',rad_list)
#     print()
#     print('labels_list',labels_list)

In [ ]:
# import pandas as pd
# df = pd.DataFrame({'BoolCol': [1, 2, 3, 3, 4],'attr': [22, 33, 22, 44, 66]},  
#        index=[10,20,30,40,50])  
# print(df)  
# a = df[(df.BoolCol==3)&(df.attr==22)].index.tolist()  
# print(a)  

In [ ]:
def Calculate_IoU(predicted_bound, ground_truth_bound):
    pxmin, pymin, pxmax, pymax = predicted_bound
    print("预测框P的坐标是：({}, {}, {}, {})".format(pxmin, pymin, pxmax, pymax))
    gxmin, gymin, gxmax, gymax = ground_truth_bound
    print("原标记框G的坐标是：({}, {}, {}, {})".format(gxmin, gymin, gxmax, gymax))
    
#  """
#  computing the IoU of two boxes.
#  Args:
#   box: (xmin, ymin, xmax, ymax),通过左下和右上两个顶点坐标来确定矩形位置
#  Return:
#   IoU: IoU of box1 and box2.
#  """
    

    parea = (pxmax - pxmin) * (pymax - pymin) # 计算P的面积
    garea = (gxmax - gxmin) * (gymax - gymin) # 计算G的面积
    print("预测框P的面积是：{}；原标记框G的面积是：{}".format(parea, garea))
    print('mark1')

    # 求相交矩形的左下和右上顶点坐标(xmin, ymin, xmax, ymax)
    xmin = max(pxmin, gxmin) # 得到左下顶点的横坐标
    ymin = max(pymin, gymin) # 得到左下顶点的纵坐标
    xmax = min(pxmax, gxmax) # 得到右上顶点的横坐标
    ymax = min(pymax, gymax) # 得到右上顶点的纵坐标
    print('mark2')
    # 计算相交矩形的面积
    w = xmax - xmin
    h = ymax - ymin
    if w <=0 or h <= 0:
        return 0,(0,0,0,0)
    print('mark3')
    area = w * h # G∩P的面积
    # area = max(0, xmax - xmin) * max(0, ymax - ymin) # 可以用一行代码算出来相交矩形的面积
    print("G∩P的坐标是：",xmin,ymin,xmax,ymax)
    print("G∩P的面积是：{}".format(area))

    # 并集的面积 = 两个矩形面积 - 交集面积
    IoU = area / (parea + garea - area)

    return IoU,(xmin,ymin,xmax,ymax)
 
if __name__ == '__main__':
    IoU = Calculate_IoU( (-1, -1, 1, 1), (0, 0, 2, 2))
    print("IoU是：{}".format(IoU))

In [ ]:
# import numpy as np
import pandas as pd

from tqdm import tqdm
from ensemble_boxes import *

consist_iou_threshold=0.5
# ===============================
# Default WBF config (you can change these)
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1
# ===============================

# Loading the train DF
# df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
df = train_df
df.fillna(0, inplace=True)
# df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

results = []
image_ids = df["image_id"].unique()


# count=3000#


for image_id in tqdm(image_ids, total=len(image_ids)):
#     count-=1
#     print('count',count)
#     if count==0:#
#         break#
    
    print('image_id',image_id)
        
    # All annotations for the current image.
    data = df[df["image_id"] == image_id]
    data = data.reset_index(drop=True)
    annotations = {}
    weights = []
    
    
    width=data.iloc[0].width
    height=data.iloc[0].height
    image_path=data.iloc[0].image_path
    class_name=data.iloc[0].class_name

    # WBF expects the coordinates in 0-1 range.
    max_value = data.iloc[:, 4:8].values.max()
    data.loc[:, ["x_min", "y_min", "x_max", "y_max"]] = data.iloc[:, 4:8]

    # Loop through all of the annotations
    for idx, row in data.iterrows():
#         print('index',row.index.tolist()[0]) 
        class_name_id = row["class_name"]
#         print('index',row.index)
        if class_name_id not in annotations:
            annotations[class_name_id] = {
                "boxes_list": [],
                "rad_id_list": [],
                "labels_list": [],
            }

            # We consider all of the radiologists as equal.
            weights.append(1.0)

        annotations[class_name_id]["boxes_list"].append([row["x_min"], row["y_min"], row["x_max"], row["y_max"]])
        annotations[class_name_id]["rad_id_list"].append(row["rad_id"])
        annotations[class_name_id]["labels_list"].append(class_name_id)
        print('class_name_id',class_name_id)

    boxes_list = []
    rad_list = []
    labels_list = []
    
    print('annotations',annotations)

    for annotator in annotations.keys():
        boxes_list.append(annotations[annotator]["boxes_list"])
        rad_list.append(annotations[annotator]["rad_id_list"])
        labels_list.append(annotations[annotator]["labels_list"])
        
    print()
    print('boxes_list',boxes_list)
    print()
    print('rad_list',rad_list)
    print()
    print('labels_list',labels_list)
    print()
    print()
    
#     for i in range(len(boxes_list)):
        

#         for j in range(0,len(boxes_list[i])-1):
#             iou_value_list=[]
# #             G_P_box_list=[]
# #             box_pass_sign=False
#             for k in range(1+j,len(boxes_list[i])):
#                 gt_ground=(boxes_list[i][j][0], boxes_list[i][j][1], boxes_list[i][j][2], boxes_list[i][j][3])
#                 pr_ground=(boxes_list[i][k][0], boxes_list[i][k][1], boxes_list[i][k][2], boxes_list[i][k][3])
#                 print('gt_ground',gt_ground)
#                 print('pr_ground',pr_ground)
#                 iou_value,G_P_box=Calculate_IoU(gt_ground,pr_ground)
#                 iou_value_list.append(iou_value)
#                 # 如果两个框iou大于阈值consist_iou_threshold
#                 if iou_value>consist_iou_threshold:
#                     results.append({
#                     "image_id": image_id,
#                     "class_id": int(class_dictionary[labels_list[i][0]]),
#                     "rad_id": "wbf",
#                     "x_min": G_P_box[0],
#                     "y_min": G_P_box[1],
#                     "x_max": G_P_box[2],
#                     "y_max": G_P_box[3],
#                     "width": width,
#                     "height": height,
#                     "image_path": image_path,
#                     "class_name":  labels_list[i][0]
#                 })

    #                     G_P_box_list.append(G_P_box)
    #             for p in range(len(iou_value_list)):
    #                 pass
    #                 np.mean(iou_value_list)
    #             if box_pass_sign:

    for i in range(len(boxes_list)):
        
        if (labels_list[i][0]=='Aortic enlargement')|(labels_list[i][0]=='Cardiomegaly')|(labels_list[i][0]=='Pleural effusion')|(labels_list[i][0]=='Pleural thickening')|(labels_list[i][0]=='Pulmonary fibrosis'):
            for j in range(0,len(boxes_list[i])-1):
                iou_value_list=[]
    #             G_P_box_list=[]
    #             box_pass_sign=False
                for k in range(1+j,len(boxes_list[i])):
                    gt_ground=(boxes_list[i][j][0], boxes_list[i][j][1], boxes_list[i][j][2], boxes_list[i][j][3])
                    pr_ground=(boxes_list[i][k][0], boxes_list[i][k][1], boxes_list[i][k][2], boxes_list[i][k][3])
                    print('gt_ground',gt_ground)
                    print('pr_ground',pr_ground)
                    iou_value,G_P_box=Calculate_IoU(gt_ground,pr_ground)
                    iou_value_list.append(iou_value)
                    # 如果两个框iou大于阈值consist_iou_threshold
                    if iou_value>consist_iou_threshold:
                        results.append({
                        "image_id": image_id,
                        "class_id": int(class_dictionary[labels_list[i][0]]),
                        "rad_id": "wbf",
                        "x_min": G_P_box[0],
                        "y_min": G_P_box[1],
                        "x_max": G_P_box[2],
                        "y_max": G_P_box[3],
                        "width": width,
                        "height": height,
                        "image_path": image_path,
                        "class_name":  labels_list[i][0]
                    })

    #                     G_P_box_list.append(G_P_box)
    #             for p in range(len(iou_value_list)):
    #                 pass
    #                 np.mean(iou_value_list)
    #             if box_pass_sign:
        else:
            for j in range(0,len(boxes_list[i])-1):
                results.append({
                    "image_id": image_id,
                    "class_id": int(class_dictionary[labels_list[i][0]]),
                    "rad_id": "wbf",
                    "x_min": boxes_list[i][j][0],
                    "y_min": boxes_list[i][j][1],
                    "x_max": boxes_list[i][j][2],
                    "y_max": boxes_list[i][j][3],
                    "width": width,
                    "height": height,
                    "image_path": image_path,
                    "class_name":  labels_list[i][0]
                })

                
#             'Aortic enlargement''Cardiomegaly''Pleural effusion''Pleural thickening''Pulmonary fibrosis'
        
                
                

results = pd.DataFrame(results)

In [ ]:
results = pd.DataFrame(results)
results['class_id'].value_counts(normalize = False, dropna = False)
train_df=results
train_df

In [ ]:
train_df['class_id'].value_counts(normalize = False, dropna = False)

In [ ]:
# import numpy as np
# a = [2,4,6,8,10]
# average_a = np.mean(a)
# average_a

# Split

In [ ]:
# !pip install ensemble_boxes

In [ ]:
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

In [ ]:
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

In [ ]:
# os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
# os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
# label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'

In [ ]:
import os.path as osp
from path import Path
from collections import Counter
import cv2
from ensemble_boxes import *

In [ ]:
imagepaths = train_df['image_path'].unique()
train_annotations=train_df

In [ ]:
# img_array  = cv2.imread('/kaggle/input/vinbigdata-512-image-dataset/vinbigdata/train/d3637a1935a905b3c326af31389cb846.png')

In [ ]:
def Create_nms_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = nms(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
# iou_thr = 0.5
# skip_box_thr = 0.0001
# viz_images = []
# for i, path in tqdm(enumerate(imagepaths[5:6])):
#     image_basename = Path(path).stem
#     print(f"(\'{image_basename}\', \'{path}\')")
#     img_annotations = train_annotations[train_annotations.image_id==image_basename]

#     boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
#     labels_viz = img_annotations['class_id'].to_numpy().tolist()

#     print("Bboxes before nms:\n", boxes_viz)
#     print("Labels before nms:\n", labels_viz)

#     boxes_list = []
#     scores_list = []
#     labels_list = []
#     weights = []

#     boxes_single = []
#     labels_single = []

#     cls_ids = img_annotations['class_id'].unique().tolist()
#     count_dict = Counter(img_annotations['class_id'].tolist())
#     print(count_dict)

#     for cid in cls_ids:       
#         ## Performing Fusing operation only for multiple bboxes with the same label
#         if count_dict[cid]==1:
#             labels_single.append(cid)
#             boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

#         else:
#             cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
#             labels_list.append(cls_list)
#             bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

#             ## Normalizing Bbox by Image Width and Height
#             bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


#             bbox = np.clip(bbox, 0, 1)
#             boxes_list.append(bbox.tolist())

#             scores_list.append(np.ones(len(cls_list)).tolist())

#             weights.append(1)


#     # Perform NMS
#     boxes, scores, box_labels = nms(boxes_list, scores_list, labels_list, weights=weights,
#                                     iou_thr=iou_thr)
    
#     print("Bboxes without multipy:\n", boxes)

#     #img_array.shape[1]是宽度
#     boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
#     boxes = boxes.round(1).tolist()
#     box_labels = box_labels.astype(int).tolist()

#     boxes.extend(boxes_single)
#     box_labels.extend(labels_single)

#     print("Bboxes after nms:\n", boxes)
#     print("Labels after nms:\n", box_labels)

#     count_dict = Counter(box_labels)
#     print(count_dict)

#     text_create('./',image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def Create_softnms_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
    sigma = 0.1
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def Create_non_maximum_weighted_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def Create_weighted_boxes_fusion_box_txt(desktop_path,name):
    iou_thr = 0.5
    skip_box_thr = 0.0001
    viz_images = []
#     image_basename = Path(path).stem
    image_basename = name
#     print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = train_annotations[train_annotations.image_id==image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    print("Bboxes before nms:\n", boxes_viz)
    print("Labels before nms:\n", labels_viz)

    boxes_list = []
    scores_list = []
    labels_list = []
    weights = []

    boxes_single = []
    labels_single = []

    cls_ids = img_annotations['class_id'].unique().tolist()
    count_dict = Counter(img_annotations['class_id'].tolist())
    print(count_dict)

    for cid in cls_ids:       
        ## Performing Fusing operation only for multiple bboxes with the same label
        if count_dict[cid]==1:
            labels_single.append(cid)
            boxes_single.append(img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().squeeze().tolist())

        else:
            cls_list =img_annotations[img_annotations.class_id==cid]['class_id'].tolist()
            labels_list.append(cls_list)
            bbox = img_annotations[img_annotations.class_id==cid][['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()

            ## Normalizing Bbox by Image Width and Height
            bbox = bbox/(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])


            bbox = np.clip(bbox, 0, 1)
            boxes_list.append(bbox.tolist())

            scores_list.append(np.ones(len(cls_list)).tolist())

            weights.append(1)

            
    # Perform NMS
    if len(boxes_list)==0:
        boxes=boxes_single
        box_labels=labels_single
        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)
        
        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])
        
    else:
        boxes, scores, box_labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights,
                                    iou_thr=iou_thr)


        #img_array.shape[1]是宽度
        boxes = boxes*(img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"], img_annotations.iloc[0]["width"], img_annotations.iloc[0]["height"])
        boxes = boxes.round(1).tolist()
        box_labels = box_labels.astype(int).tolist()

        boxes.extend(boxes_single)
        box_labels.extend(labels_single)

        print("Bboxes after nms:\n", boxes)
        print("Labels after nms:\n", box_labels)

        count_dict = Counter(box_labels)
        print(count_dict)

        text_create(desktop_path,image_basename,boxes,box_labels,img_annotations.iloc[0]["width"],img_annotations.iloc[0]["height"])

In [ ]:
def text_create(desktop_path,name, boxes,box_labels,w,h):
    
    full_path = os.path.join(desktop_path, name+'.txt')  # 也可以创建一个.doc的word文档
    print('full_path',full_path)
    file = open(full_path, 'w')
    
    dw = 1. / (w)
    dh = 1. / (h)
    
    for i in range(len(boxes)):

        
        x = (boxes[i][0] + boxes[i][2]) / 2.0
        y = (boxes[i][1] + boxes[i][3]) / 2.0
        w = boxes[i][2] - boxes[i][0]
        h = boxes[i][3] - boxes[i][1]

        
        x = x * dw
        w = w * dw
        y = y * dh
        h = h * dh
        
        
        file.write(str(box_labels[i])+ ' '+ str(x)+ ' '+ str(y)+ ' '+ str(w)+ ' '+ str(h)+ ' '+ '\n') 
        print(str(box_labels[i])+ ' '+ str(x)+ ' '+ str(y)+ ' '+ str(w)+ ' '+ str(h)+ ' ')
    file.close()

# Copying Files

In [ ]:
os.makedirs('/kaggle/working/vinbigdata/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/train', exist_ok = True)
os.makedirs('/kaggle/working/vinbigdata/images/val', exist_ok = True)
label_dir = '/kaggle/input/vinbigdata-yolo-labels-dataset/labels'
for file in tqdm(train_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    
    # nms stuff
    print(filename)
    Create_softnms_box_txt('/kaggle/working/vinbigdata/labels/train',filename)
    
#     shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, '/kaggle/working/vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    
    # nms stuff
    Create_softnms_box_txt('/kaggle/working/vinbigdata/labels/val',filename)
#     shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/vinbigdata/labels/val')

In [ ]:
# import os
# path = os.getcwd()#获取当前路径
# print(path)

# # all_files = [f for f in os.listdir('/kaggle/working/vinbigdata/labels/val' )]#输出根path下的所有文件名到一个列表中
# all_files = [f for f in os.listdir(path )]#输出根path下的所有文件名到一个列表中
# #对各个文件进行处理
# print(all_files)

# Get Class Name

In [ ]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

# [YOLOv5](https://github.com/ultralytics/yolov5)
![](https://user-images.githubusercontent.com/26833433/98699617-a1595a00-2377-11eb-8145-fc674eb9b1a7.jpg)
![](https://user-images.githubusercontent.com/26833433/90187293-6773ba00-dd6e-11ea-8f90-cd94afc0427f.png)

# YOLOv5 Stuff

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = '/kaggle/working/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('/kaggle/working/vinbigdata/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 14,
    names = classes
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
# %pip install -qr requirements.txt # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/
Image(filename='runs/detect/exp/zidane.jpg', width=600)

## Pretrained Checkpoints:

| Model | AP<sup>val</sup> | AP<sup>test</sup> | AP<sub>50</sub> | Speed<sub>GPU</sub> | FPS<sub>GPU</sub> || params | FLOPS |
|---------- |------ |------ |------ | -------- | ------| ------ |------  |  :------: |
| [YOLOv5s](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 37.0     | 37.0     | 56.2     | **2.4ms** | **416** || 7.5M   | 13.2B
| [YOLOv5m](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 44.3     | 44.3     | 63.2     | 3.4ms     | 294     || 21.8M  | 39.4B
| [YOLOv5l](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | 47.7     | 47.7     | 66.5     | 4.4ms     | 227     || 47.8M  | 88.1B
| [YOLOv5x](https://github.com/ultralytics/yolov5/releases/tag/v3.0)    | **49.2** | **49.2** | **67.7** | 6.9ms     | 145     || 89.0M  | 166.4B
| | | | | | || |
| [YOLOv5x](https://github.com/ultralytics/yolov5/releases/tag/v3.0) + TTA|**50.8**| **50.8** | **68.9** | 25.5ms    | 39      || 89.0M  | 354.3B
| | | | | | || |
| [YOLOv3-SPP](https://github.com/ultralytics/yolov5/releases/tag/v3.0) | 45.6     | 45.5     | 65.2     | 4.5ms     | 222     || 63.0M  | 118.0B

# Selecting Models
In this notebok I'm using `v5s`. To select your prefered model just replace `--cfg models/yolov5s.yaml --weights yolov5s.pt` with the following command:
* `v5s` : `--cfg models/yolov5s.yaml --weights yolov5s.pt`
* `v5m` : `--cfg models/yolov5m.yaml --weights yolov5m.pt`
* `v5l` : `--cfg models/yolov5l.yaml --weights yolov5l.pt`
* `v5x` : `--cfg models/yolov5x.yaml --weights yolov5x.pt`

# Train

In [ ]:
# !WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache 
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 50 --data /kaggle/working/vinbigdata.yaml --weights yolov5s.pt --cache

# Class Distribution

In [ ]:
a=1

In [ ]:
plt.figure(figsize = (20,20))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels_correlogram.jpg'));

In [ ]:
plt.figure(figsize = (20,20))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels.jpg'));

# Batch Image

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch0.jpg'))

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch1.jpg'))

plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch2.jpg'))

# GT Vs Pred

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (2*5,3*5), constrained_layout = True)
for row in range(3):
    ax[row][0].imshow(plt.imread(f'runs/train/exp/test_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'runs/train/exp/test_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'runs/train/exp/test_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'runs/train/exp/test_batch{row}_pred.jpg', fontsize = 12)

# (Loss, Map) Vs Epoch

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/results.png'));

# Confusion Matrix

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/confusion_matrix.png'));

In [ ]:
# 0     3625
# 3     2860
# 13    1299
# 11     763 0.5
# 8      728
# 10     699
# 7      448
# 9      357 0.67
# 6      291
# 5      290
# 2      193
# 4      121
# 12      97
# 1       46

# Inference

In [ ]:
!python detect.py --weights 'runs/train/exp/weights/best.pt'\
--img 640\
--conf 0.15\
--iou 0.5\
--source /kaggle/working/vinbigdata/images/val\
--exist-ok

# Inference Plot

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*')
for _ in range(3):
    row = 4
    col = 4
    grid_files = random.sample(files, row*col)
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
a=1

In [ ]:
# shutil.rmtree('/kaggle/working/vinbigdata')
# shutil.rmtree('runs/detect')
# for file in (glob('runs/train/exp/**/*.png', recursive = True)+glob('runs/train/exp/**/*.jpg', recursive = True)):
#     os.remove(file)

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test'
# weights_dir = '/kaggle/input/vinbigdata-cxr-ad-yolov5-14-class-train/yolov5/runs/train/exp/weights/best.pt'
weights_dir = f'runs/train/exp/weights/best.pt'

In [ ]:
test_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test.csv')
test_df.head()

In [ ]:
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!python detect.py --weights $weights_dir\
--img 640\
--conf 0.15\
--iou 0.4\
--source $test_dir\
--save-txt --save-conf --exist-ok

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
#     print('bboxes',bboxes)
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

In [ ]:
pred_df_pure = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df_pure = pd.merge(test_df, pred_df_pure, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df_pure = sub_df_pure[['image_id', 'PredictionString']]
sub_df_pure.to_csv('/kaggle/working/sub_df_pure.csv',index = False)
sub_df_pure.tail()

In [ ]:
display(sub_df_pure)

In [ ]:
a=1

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random
import cv2
from glob import glob
from tqdm import tqdm

files = glob('runs/detect/exp/*png')
for _ in range(6):
    row = 4
    col = 4
    grid_files = random.sample(files, row*col)
    images     = []
    for image_path in tqdm(grid_files):
        img          = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        images.append(img)

    fig = plt.figure(figsize=(col*5, row*5))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(col, row),  # creates 2x2 grid of axes
                     axes_pad=0.05,  # pad between axes in inch.
                     )

    for ax, im in zip(grid, images):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

dim = 512 #512, 256, 'original'
fold = 4

train_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/train.csv')
train_df.head()

train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()

In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

In [ ]:
list_mid_gaussian_kde=[]
for i in range(14):
    

    train_df_0 = train_df[train_df.class_id==i]
    x_val = train_df_0.x_mid
    y_val = train_df_0.y_mid
    print(len(x_val))

    # Calculate the point density
    xy = np.vstack([x_val,y_val])
    z = gaussian_kde(xy)(xy)

    fig, ax = plt.subplots(figsize = (10, 10))
    # ax.axis('off')
    ax.axis([0,1,1,0])
    ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
    # ax.set_xlabel('x_mid')
    # ax.set_ylabel('y_mid')
    plt.show()

    evalutor=gaussian_kde(xy)
    # evalutor.evaluate([[0.6],[0.4]])
    list_mid_gaussian_kde.append(evalutor)

In [ ]:
list_w_h_gaussian_kde=[]
for i in range(14):
    

    train_df_0 = train_df[train_df.class_id==i]
    x_val = train_df_0.w
    y_val = train_df_0.h
    print(len(x_val))

    # Calculate the point density
    xy = np.vstack([x_val,y_val])
    z = gaussian_kde(xy)(xy)

    fig, ax = plt.subplots(figsize = (10, 10))
    # ax.axis('off')
    ax.axis([0,1,1,0])
    ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
    # ax.set_xlabel('x_mid')
    # ax.set_ylabel('y_mid')
    plt.show()

    evalutor=gaussian_kde(xy)
    # evalutor.evaluate([[0.6],[0.4]])
    list_w_h_gaussian_kde.append(evalutor)

In [ ]:
# https://www.pythonheidong.com/blog/article/436765/a0facf9464d9337dc1eb/
list_area_gaussian_kde=[]
for i in range(14):
    train_df_0 = train_df[train_df.class_id==i]
    data = train_df_0.w

    density = gaussian_kde(data)
    xs = np.linspace(0,1,200)
    plt.plot(xs,density(xs))
    plt.show()
    list_area_gaussian_kde.append(density)

In [ ]:
def Hotmask_mid_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_mid_gaussian_kde[int(class_id)]
    value = evaluator.evaluate([[normalized_xmid_ymid_w_h[0]],[normalized_xmid_ymid_w_h[1]]])
    
    return value

def Hotmask_w_h_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_w_h_gaussian_kde[int(class_id)]
    value = evaluator.evaluate([[normalized_xmid_ymid_w_h[2]],[normalized_xmid_ymid_w_h[3]]])
    
    return value

def Hotmask_area_value(class_id,normalized_xmid_ymid_w_h):
    evaluator = list_area_gaussian_kde[int(class_id)]
    value = evaluator.evaluate(normalized_xmid_ymid_w_h[2]*normalized_xmid_ymid_w_h[3])
    
    return value

In [ ]:
image_ids = []
PredictionStrings = []
sum_delte_on=0

# threshold=0.001
threshold_mid=0.001
threshold_w_h=0.001
threshold_area=0.001

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    delte_on=0
    
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
#     print('data',data.type)
    
    list_classid_probility = data[:, :2]
    list_normalized_box = data[:, 2:]
    delte_sign=[]
#     filtered_list_classid_probility=[]
#     filtered_list_normalized_box=[]
    
#     print('data[:, :2]',data[:, :2]) 
    
    for i in range(len(list_classid_probility)):
        if (Hotmask_mid_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_mid and Hotmask_w_h_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_w_h and Hotmask_area_value(list_classid_probility[i][0] , list_normalized_box[i])>threshold_area) or list_classid_probility[i][0]==14:
            delte_sign.append(True)
        else:
            delte_sign.append(False)
            delte_on+=1
            
#             filtered_list_classid_probility.append(list_classid_probility[i][0])
#             filtered_list_normalized_box.append(list_classid_probility[i].tolist())
            
#     prin**t('filtered_list_normalized_box',filtered_list_normalized_box) 
    
    print('delte_sign',delte_sign) 
    bboxes = list(np.round(np.concatenate((data[:, :2][delte_sign], np.round(yolo2voc(h, w, data[:, 2:][delte_sign]))), axis =1).reshape(-1), 1).astype(str))
#     bboxes = np.array(list(np.round(np.concatenate((filtered_list_classid_probility, np.round(yolo2voc(h, w, filtered_list_normalized_box))), axis =1).reshape(-1), 1).astype(str)),dtype=np.float32)
#     print('data[:, 2:]',data[:, 2:])
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))
    
    if delte_on!=0:
        sum_delte_on+=1

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]

sub_df.loc[sub_df['PredictionString'] =='','PredictionString']="14 1 0 0 1 1"
sub_df.to_csv('/kaggle/working/submission_softnms.csv',index = False)
sub_df.tail()

In [ ]:
display(sub_df)

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import shutil

In [ ]:
import os 
os.getcwd()

In [ ]:
pred_14cls =sub_df
pred_2cls = pd.read_csv('/kaggle/input/vinbigdata-2class-prediction/2-cls test pred.csv')

In [ ]:
count_low=0
count_mid=0
count_high=0
def filter_2cls_raw(row, low_thr=0.08, high_thr=0.95):
    global count_low
    global count_mid
    global count_high
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
        count_low+=1
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
        count_mid+=1
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
        count_high+=1
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [ ]:
pred_raw = pd.merge(pred_14cls, pred_2cls, on = 'image_id', how = 'left')
pred_raw.head()

In [ ]:
sub_raw = pred_raw.apply(filter_2cls_raw, axis=1)
print(count_low/3000,count_mid/3000,count_high/3000)

In [ ]:
sub_raw[['image_id', 'PredictionString']].to_csv('submission_raw.csv',index = False)

In [ ]:
print_df=sub_raw[['image_id', 'PredictionString']]

In [ ]:
display(print_df)

In [ ]:
# for i in range(sub_raw.shape[0]):
# #     sub_raw.iloc[i]['image_id', 'PredictionString']
#     print(print_df.loc[i])